In [ ]:
%matplotlib inline
import os
os.environ["THEANO_FLAGS"] = "device=gpu"
import theano
from caffezoo.googlenet import GoogleNet
from caffezoo.vgg import VGG
from skimage.io import imread
from skimage.transform import AffineTransform, resize, SimilarityTransform, warp
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.base import clone
from sklearn.manifold import TSNE, LocallyLinearEmbedding, Isomap
from skimage.util import pad
from sklearn.cross_validation import cross_val_score

In [ ]:
images = [
    #imread('dog_PNG178.png'),
    #imread('forest.jpg'),
    imread('image1.jpg'),
]

In [ ]:
def concat(layers):
    layers = [layer.reshape((layer.shape[0], np.prod(layer.shape[1:]))) for layer in layers]
    return np.concatenate(layers, axis=1)

def gram_matrix(x):
    g = (x[:, :, None, :, :] * x[:, None, :, :, :]).sum(axis=(3, 4))
    return g.reshape((x.shape[0],-1))
    #x = x.reshape((x.shape[0], x.shape[1], -1))
    #g = np.tensordot(x, x, axes=([2], [2]))
    #return g

def reduce_dim(f, dropout=0.5, seed=1234):
    def f_(layers):
        rng = np.random.RandomState(seed)
        layers = [
            l[:, (rng.uniform(size=l.shape[1]) > dropout)] for l in layers
        ]
        return f(layers)
    return f_
def gram(layers):
    layers = [
        gram_matrix(layer) for layer in layers
    ]
    return concat(layers)
def as_is(layers):
    return layers[0]

In [ ]:
# rescale to (w, h)
w, h = 224, 224
images = [resize(im, (w, h), preserve_range=True) for im in images]

In [ ]:
L = [ "input",
      "conv2/3x3_reduce",
     "inception_3a/output", "inception_3b/output", 
     "inception_4a/output", "inception_4b/output",
     "inception_4c/output", "inception_4d/output",
     "inception_4e/output", "inception_5a/output",
     "inception_5b/output", "inception_5b/output", "pool5/7x7_s1"]
layers = [L[2]]
model = GoogleNet(resize=False, 
                  layer_names=layers, 
                  aggregate_function=concat, 
                  batch_size=50)
model._load()

In [ ]:
transformed_images = np.array(images)

In [ ]:
model.transform(transformed_images)